In [79]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
%matplotlib inline

In [85]:
#apple_csv = Path("..data/csv_files/Apple.csv")
apple_df = pd.read_csv("../data/csv_files/Apple.csv", index_col= "Date", parse_dates=True, infer_datetime_format=True)

facebook_df = pd.read_csv("../data/csv_files/Facebook.csv", index_col= "Date", parse_dates=True, infer_datetime_format=True)

google_df = pd.read_csv("../data/csv_files/Google.csv", index_col= "Date", parse_dates=True, infer_datetime_format=True)

netflix_df = pd.read_csv("../data/csv_files/Netflix.csv", index_col= "Date", parse_dates=True, infer_datetime_format=True)

amazon_df = pd.read_csv("../data/csv_files/Amazon.csv", index_col= "Date", parse_dates=True, infer_datetime_format=True)

In [86]:
apple_df.rename(columns={"Close":"AAPL"}, inplace=True)

google_df.rename(columns={"Close":"GOOGL"}, inplace=True)

netflix_df.rename(columns={"Close":"NFLX"}, inplace=True)

facebook_df.rename(columns={"Close":"FB"}, inplace=True)

amazon_df.rename(columns={"Close":"AMZN"}, inplace=True)

In [87]:
apple_df.head()

,AAPL
Date,
2019-10-29,60.82
2019-10-30,60.82
2019-10-31,62.19
2019-11-01,63.96
2019-11-04,64.38


In [90]:
daily_prices = pd.concat([facebook_df, apple_df, amazon_df, netflix_df, google_df], axis="columns", join="inner")

daily_prices.sort_index(inplace=True)
daily_prices.head()

,FB,AAPL,AMZN,NFLX,GOOGL
Date,,,,,
2019-10-29,189.31,60.82,1762.71,281.21,1260.66
2019-10-30,188.25,60.82,1779.99,291.45,1260.70
2019-10-31,191.65,62.19,1776.66,287.41,1258.80
2019-11-01,193.62,63.96,1791.44,286.81,1272.25
2019-11-04,194.72,64.38,1804.66,292.86,1289.61


In [91]:
daily_prices.reset_index()

,Date,FB,AAPL,AMZN,NFLX,GOOGL
0,2019-10-29,189.31,60.82,1762.71,281.21,1260.66
1,2019-10-30,188.25,60.82,1779.99,291.45,1260.70
2,2019-10-31,191.65,62.19,1776.66,287.41,1258.80
3,2019-11-01,193.62,63.96,1791.44,286.81,1272.25
4,2019-11-04,194.72,64.38,1804.66,292.86,1289.61
...,...,...,...,...,...,...
248,2020-10-22,278.12,115.75,3176.40,485.23,1606.66
249,2020-10-23,284.79,115.04,3204.40,488.28,1632.98
250,2020-10-26,277.11,115.05,3207.04,488.24,1584.29
251,2020-10-27,283.29,116.60,3286.33,488.93,1598.88
